In [119]:
import matplotlib as plt
import sys
import pandas as pd
import numpy as np

sys.path.append(r"C:\Users\louis\OneDrive\Documents\CS\BCG Datathon\Datathon_BCG_colissiMONSTRE\datathon_bcg\module")
sys.path.append(r"C:\Users\louis\OneDrive\Documents\CS\BCG Datathon\Datathon_BCG_colissiMONSTRE\datathon_bcg\data\csv\champs")

from datetime import datetime
import pytz

from dataprep import traiter_donnees
from dataprep import completer_heures_manquantes

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
df=pd.read_csv(r"C:\Users\louis\OneDrive\Documents\CS\BCG Datathon\Datathon_BCG_colissiMONSTRE\datathon_bcg\data\csv\champs\champs_2023.csv", delimiter=';')

In [4]:
df=traiter_donnees(df)

In [52]:
df

,Libelle,taux_occupation,etat_arc,debit_horaire
timestamp,,,,
2023-01-01 00:00:00,AV_Champs_Elysees,NaN,Barré,3.0
2023-01-01 01:00:00,AV_Champs_Elysees,NaN,Barré,116.0
2023-01-01 02:00:00,AV_Champs_Elysees,34.44445,Invalide,534.0
2023-01-01 03:00:00,AV_Champs_Elysees,25.47389,Invalide,622.0
2023-01-01 04:00:00,AV_Champs_Elysees,26.02722,Invalide,756.0
...,...,...,...,...
2023-11-30 20:00:00,AV_Champs_Elysees,30.87000,Invalide,1433.0
2023-11-30 21:00:00,AV_Champs_Elysees,24.38111,Invalide,1253.0
2023-11-30 22:00:00,AV_Champs_Elysees,20.40723,Invalide,1069.0


In [16]:
# Compter les valeurs manquantes dans la colonne spécifiée
nombre_de_donnees_manquantes = df["taux_occupation"].isnull().sum()/len(df)

print(f"Le nombre de données manquantes dans la colonne '{'taux_occupation'}' est : {nombre_de_donnees_manquantes}")

nombre_de_donnees_manquantes = df["debit_horaire"].isnull().sum()/len(df)

print(f"Le nombre de données manquantes dans la colonne '{'debit_horaire'}' est : {nombre_de_donnees_manquantes}")

Le nombre de données manquantes dans la colonne 'taux_occupation' est : 0.016755521706016754
Le nombre de données manquantes dans la colonne 'debit_horaire' est : 0.014597613607514598


# Etude de la correlation avec la meteo 

Data meteo historique récupérée : https://public.opendatasoft.com/explore/dataset/donnees-synop-essentielles-omm/export/?sort=date&q.timerange.date=date:%5B2021-08-31T22:00:00Z+TO+2023-12-15T22:59:59Z%5D&q=Paris

Data meteo prediction : https://www.pleinchamp.com/meteo/heure-par-heure/75001-paris/2023-12-03


In [55]:
meteo=pd.read_csv(r'C:\Users\louis\OneDrive\Documents\CS\BCG Datathon\Datathon_BCG_colissiMONSTRE\datathon_bcg\data\meteo.csv', delimiter=';')

In [35]:
meteo.head()

,ID OMM station,Date,Pression au niveau mer,Variation de pression en 3 heures,Type de tendance barométrique,Direction du vent moyen 10 mn,Vitesse du vent moyen 10 mn,Température,Point de rosée,Humidité,...,Altitude,communes (name),communes (code),EPCI (name),EPCI (code),department (name),department (code),region (name),region (code),mois_de_l_annee
0,7149,2021-09-24T02:00:00+02:00,102100,-10.0,0,310,0.5,286.75,284.65,87,...,89,Athis-Mons,91027,Métropole du Grand Paris,200054781,Essonne,91,Île-de-France,11,9
1,7149,2021-09-21T23:00:00+02:00,103010,40.0,3,10,2.7,287.15,283.55,79,...,89,Athis-Mons,91027,Métropole du Grand Paris,200054781,Essonne,91,Île-de-France,11,9
2,7149,2021-09-22T02:00:00+02:00,103010,-10.0,8,360,0.9,285.05,282.75,86,...,89,Athis-Mons,91027,Métropole du Grand Paris,200054781,Essonne,91,Île-de-France,11,9
3,7149,2021-10-11T20:00:00+02:00,102900,50.0,3,360,2.2,285.55,280.05,69,...,89,Athis-Mons,91027,Métropole du Grand Paris,200054781,Essonne,91,Île-de-France,11,10
4,7149,2021-10-12T02:00:00+02:00,102840,-50.0,8,330,2.2,281.75,280.05,89,...,89,Athis-Mons,91027,Métropole du Grand Paris,200054781,Essonne,91,Île-de-France,11,10


In [123]:
def clean_meteo(df):


    # Réduire le DataFrame aux colonnes spécifiées
    nouveau_data = df.loc[:, ['Date', 'Précipitations dans la dernière heure', 'Température (°C)']]
    nouveau_data.dropna(subset=['Date'], inplace=True)

    for i in range (len(nouveau_data)):
        # Convertir la chaîne en objet datetime
        date_time_obj = datetime.fromisoformat(nouveau_data['Date'][i])

        # Formatter la date en chaîne au nouveau format AAAA-MM-JJ HH:MM:SS
        nouveau_format_date = date_time_obj.strftime('%Y-%m-%d %H:%M:%S')

        nouveau_data['Date'][i]=nouveau_format_date
    nouveau_data['Date'] = pd.to_datetime(nouveau_data['Date'])
    nouveau_data = nouveau_data.sort_values(by='Date')
    
        #renommer les colonnes pour que ce soit pratique
    nouveau_data = nouveau_data.rename(columns={'Date': 'timestamp'})


    nouveau_data['timestamp'] = pd.to_datetime(nouveau_data['timestamp'], utc = True)
    nouveau_data['timestamp'] = nouveau_data['timestamp'].dt.tz_convert('Europe/Paris')
    nouveau_data['timestamp'] = nouveau_data['timestamp'].dt.tz_localize(None)
    nouveau_data.set_index('timestamp', inplace=True)
    nouveau_data=completer_heures_manquantes(nouveau_data)

    nouveau_data['MaColonne_Modifiee'] = nouveau_data['Précipitations dans la dernière heure']
    # Remplacer les valeurs manquantes par NaN pour ne considérer que ces valeurs
    nouveau_data.loc[nouveau_data['MaColonne_Modifiee'].notnull(), 'MaColonne_Modifiee'] = np.nan

    # Interpoler les valeurs manquantes en utilisant les méthodes de remplissage existantes (pad/ffill et backfill/bfill)
    nouveau_data['MaColonne_Modifiee'] = nouveau_data['MaColonne_Modifiee'].fillna(method='pad')
    nouveau_data['MaColonne_Modifiee'] = nouveau_data['MaColonne_Modifiee'].fillna(method='backfill')

    return nouveau_data

In [124]:
meteo_clean=clean_meteo(meteo)

C:\Users\louis\AppData\Local\Temp\ipykernel_19320\2917362699.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nouveau_data['Date'][i]=nouveau_format_date


In [125]:
meteo_clean

,Précipitations dans la dernière heure,Température (°C),MaColonne_Modifiee
2021-09-01 04:00:00,0.0,16.0,NaN
2021-09-01 05:00:00,NaN,NaN,NaN
2021-09-01 06:00:00,NaN,NaN,NaN
2021-09-01 07:00:00,0.0,14.5,NaN
2021-09-01 08:00:00,NaN,NaN,NaN
...,...,...,...
2023-12-01 16:00:00,NaN,NaN,NaN
2023-12-01 17:00:00,0.0,2.5,NaN
2023-12-01 18:00:00,NaN,NaN,NaN
2023-12-01 19:00:00,NaN,NaN,NaN


In [103]:
meteo_clean

,Précipitations dans la dernière heure,Température
timestamp,,
2021-09-01 04:00:00,0.0,289.15
2021-09-01 07:00:00,0.0,287.65
2021-09-01 10:00:00,0.0,287.95
2021-09-01 13:00:00,0.0,291.25
2021-09-01 16:00:00,0.0,295.05
...,...,...
2023-12-01 08:00:00,0.0,276.65
2023-12-01 11:00:00,0.0,276.45
2023-12-01 14:00:00,0.0,276.05


In [66]:
date_initiale = meteo['Date'][0]

# Convertir la chaîne en objet datetime
date_time_obj = datetime.fromisoformat(date_initiale)

# Formatter la date en chaîne au nouveau format AAAA-MM-JJ HH:MM:SS
nouveau_format_date = date_time_obj.strftime('%Y-%m-%d %H:%M:%S')
meteo_clean[]
print("Date convertie :", nouveau_format_date)

Date convertie : 2021-09-24 02:00:00


In [65]:
meteo_clean

,Date,Précipitations dans la dernière heure,Température
0,2021-09-24T02:00:00+02:00,0.0,286.75
1,2021-09-21T23:00:00+02:00,0.0,287.15
2,2021-09-22T02:00:00+02:00,0.0,285.05
3,2021-10-11T20:00:00+02:00,0.0,285.55
4,2021-10-12T02:00:00+02:00,0.0,281.75
...,...,...,...
6554,2022-08-10T11:00:00+02:00,0.0,299.25
6555,2022-07-28T20:00:00+02:00,0.0,298.75
6556,2022-07-31T14:00:00+02:00,0.0,299.65
6557,2022-06-23T17:00:00+02:00,0.0,300.15


In [57]:
meteo_clean['Date'] = pd.to_datetime(data['Date'])

# Ordonner le DataFrame en fonction de la colonne 'Date'
data = data.sort_values(by='Date')

# Afficher le DataFrame trié
print(data)

KeyError: 0

In [54]:
clean_meteo['Date'] = datetime.fromisoformat(clean_meteo['Date'])

TypeError: 'function' object is not subscriptable

In [53]:
date_initiale = meteo['Date'][0]

# Convertir la chaîne en objet datetime
date_time_obj = datetime.fromisoformat(date_initiale)

# Formatter la date en chaîne au nouveau format AAAA-MM-JJ HH:MM:SS
nouveau_format_date = date_time_obj.strftime('%Y-%m-%d %H:%M:%S')

print("Date convertie :", nouveau_format_date)

Date convertie : 2021-09-24 02:00:00


In [33]:
meteo['Date'][0]

'2021-09-24T02:00:00+02:00'